In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "GPU-cfc996e3-3993-d092-228c-0d39c868bf05"

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.models.resnet import resnet50, resnet18
from torch.utils.data import DataLoader, random_split
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torchvision.transforms import ToTensor, Compose, Normalize
from tqdm.notebook import tqdm 
import torch

from neossim.pipeline.img import get_dataset_mean_std


dataset = ImageFolder("/home/ki/tmp/datasets/imagenet-2010-openset-32x32", transform=ToTensor())
num_classes = 360

model = resnet18(pretrained=False, num_classes=num_classes)

size_train = int(0.9 * len(dataset))
size_test = int(len(dataset) - size_train)
ds_train, ds_val = random_split(dataset, lengths=[size_train, size_test])
train_loader = DataLoader(ds_train, shuffle=True, num_workers=10, batch_size=128)
val_loader = DataLoader(ds_val, num_workers=10, batch_size=512)
criterion =  CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [ ]:
# mean, std = get_dataset_mean_std(train_loader)
# transform = Compose([ToTensor(), Normalize(mean=mean, std=std)])
# dataset.transforms = transform

In [ ]:
model.cuda()

bar1 = tqdm(total=len(train_loader))
bar2 = tqdm(total=len(val_loader))

train_acc = []
val_acc = []

for epoch in range(100):
    print(f"Starting epoch {epoch}")
    # train one epoch 
    model.train()
    correct = torch.tensor(0)
    total= torch.tensor(0)
    bar1.reset()
    bar1.set_description(f"Epoch {epoch}")
    bar1.refresh()
    
    for n, batch in enumerate(train_loader):
        x,y = batch
        x = x.cuda()
        y = y.cuda()

        optimizer.zero_grad()
        prediction = model(x)
        loss = criterion(prediction, y)
        loss.backward()
        optimizer.step()

        y_hat = prediction.argmax(dim=1)
        correct += (y_hat == y).sum().item()
        total += x.shape[0]
        bar1.set_postfix({"Accuracy": (correct / total).item(), "Loss": loss.item()})
        bar1.update()

    train_acc.append((correct / total).item())

    # test 
    model.eval()
    correct = torch.tensor(0)
    total= torch.tensor(0)
    bar2.reset()
    bar2.set_description(f"Epoch {epoch}")
    bar2.refresh()
    
    with torch.no_grad():
        for n, batch in enumerate(val_loader):
            x,y = batch
            x = x.cuda()
            y = y.cuda()

            prediction = model(x)
            y_hat = prediction.argmax(dim=1)

            y_hat = prediction.argmax(dim=1)
            correct += (y_hat == y).sum().item()
            total += x.shape[0]
            bar2.set_postfix({"Accuracy": (correct / total).item(), "Loss": loss.item()})
            bar2.update()
    
    val_acc.append((correct / total).item())

    path = f"trained-model-resnet-18-32x32-{epoch}.pt"
    print(f"==> Epoch {epoch}: Train {train_acc[-1]:.2%} Val {val_acc[-1]:.2%} -> {path}")
    torch.save(model.state_dict(), path)

In [ ]:
torch.save(model.state_dict(), "trained-model-32x32.pt")

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(train_acc, label="train")
plt.plot(val_acc, label="val")